In [1]:
# Load coding libraries
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import mean_squared_error

In [4]:
# If error occur when loads autogluon package, exccute below commend to fix the problem
from tqdm import tqdm

In [5]:
# For any problems related to download autogluon please refer to https://auto.gluon.ai/stable/install.html
# For ARM(Apple M Chips), please use conda comment to install autogluon 
from autogluon.tabular import TabularDataset, TabularPredictor

In [7]:
data = TabularDataset('/Users/main/Desktop/dsci510/project/meta_data2')
len(data)

2555

In [8]:
# Load combined 5 seasons records, salary is adjusted for inflation
data = TabularDataset('/Users/main/Desktop/dsci510/project/meta_data2')
# Drop unnecessary column and prepare for training
data = data.drop(columns=['Player_y']).drop(columns=['first_name']).drop(columns=['last_name']).drop(columns=['Player_x'])
# Random split data into two groups. 80% for trainning and 20% for test/validation. Set seed for reproducible results
train_data, val_data = train_test_split(
    data, test_size=0.2, shuffle=True, random_state = 25
)
# For validation and test group, we drop predicting columns to isolate model and answer key.  
val_data_noLabel = val_data.drop(columns=['salary'])
data.head()

,salary,Team,Age,GP,W,L,Min,PTS,FGM,FGA,...,REB,AST,TOV,STL,BLK,PF,FP,DD2,TD3,+/-
0,51915615.0,GSW,35,15,8,7,33.0,29.7,9.2,19.0,...,4.6,3.9,3.7,0.8,0.1,1.7,40.3,0.0,0.0,-1.1
1,47649433.0,PHX,35,15,9,6,36.9,31.4,10.7,20.1,...,7.1,5.5,4.0,0.7,1.2,1.9,49.9,3.0,0.0,5.1
2,47607350.0,DEN,28,16,10,6,33.7,28.2,11.0,19.4,...,13.5,8.9,3.1,1.2,0.8,2.4,60.6,15.0,6.0,6.1
3,47607350.0,LAL,38,16,9,7,33.9,25.5,9.5,17.0,...,8.0,6.6,3.8,1.5,0.8,1.3,48.0,4.0,1.0,5.6
4,47607350.0,PHI,29,15,11,4,34.8,32.1,10.4,21.1,...,11.3,6.3,3.7,1.0,1.9,3.1,60.2,11.0,0.0,5.8


In [9]:
# Last check training data
data[::150]

,salary,Team,Age,GP,W,L,Min,PTS,FGM,FGA,...,REB,AST,TOV,STL,BLK,PF,FP,DD2,TD3,+/-
0,51915615.0,GSW,35,15,8,7,33.0,29.7,9.2,19.0,...,4.6,3.9,3.7,0.8,0.1,1.7,40.3,0.0,0.0,-1.1
150,9770904.0,POR,19,6,3,3,26.4,7.8,3.2,9.8,...,2.3,5.0,3.8,0.5,0.2,4.0,16.3,0.0,0.0,-5.2
300,2815937.0,PHX,25,16,10,6,21.9,6.9,2.1,4.9,...,3.2,1.1,0.8,0.9,0.4,1.1,15.3,0.0,0.0,1.1
450,559782.0,PHX,24,3,2,1,3.5,2.0,1.0,2.0,...,1.0,1.0,0.0,0.0,0.3,0.3,5.7,0.0,0.0,-2.0
600,10866131.0,DET,21,12,3,9,33.3,19.9,7.8,18.7,...,6.2,6.0,3.3,0.8,0.6,2.8,37.3,3.0,0.0,-5.0
750,2995641.0,TOR,23,9,6,3,5.5,1.0,0.3,1.6,...,0.4,0.3,0.1,0.0,0.0,0.3,1.9,0.0,0.0,0.2
900,1043199.0,DET,26,40,10,30,16.1,7.0,2.5,5.7,...,2.8,0.7,0.9,0.7,0.1,2.3,12.8,0.0,0.0,-1.0
1050,19989032.0,IND,31,34,20,14,28.5,13.1,4.4,12.1,...,4.1,6.6,2.6,1.4,0.2,2.2,30.3,5.0,0.0,5.4
1200,4743673.0,PHX,26,6,4,2,3.7,0.7,0.2,0.3,...,0.8,0.3,0.7,0.0,0.0,0.2,1.5,0.0,0.0,2.5
1350,1941874.0,MIL,29,48,26,22,9.9,3.6,1.5,2.7,...,2.1,0.5,0.5,0.3,0.3,1.4,8.0,0.0,0.0,-2.4


In [18]:
# Peak into statistic spread of our preidicting value
label = 'salary'
print("Player Salary Distribution: \n", train_data[label].describe())

Summary of Salary: 
 count    2.044000e+03
mean     8.378097e+06
std      1.082329e+07
min      6.022000e+03
25%      1.704661e+06
50%      3.351582e+06
75%      1.095707e+07
max      5.191562e+07
Name: salary, dtype: float64


In [11]:
# Before training, please change path to your desired directory. (stores trained model)
save_path = '/Users/main/Desktop/dsci510/project/model' 

In [12]:
# Please check and adjust model parameters to fit your need. Higher accuracy requires longer time and higher computing power
# Training time limit to 7 minuates
time_limit = 20*60
predictor = TabularPredictor(label=label, path=save_path, eval_metric = "mean_absolute_error").fit(train_data, time_limit=time_limit,auto_stack=True, presets= "best_quality")

Presets specified: ['best_quality']
/Users/main/anaconda3/envs/autogluon/lib/python3.10/site-packages/autogluon/core/utils/utils.py:564: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context("mode.use_inf_as_na", True):  # treat None, NaN, INF, NINF as NA
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 1200s
AutoGluon will save models to "/Users/main/Desktop/dsci510/project/model"
AutoGluon Version:  0.8.2
Python Version:     3.10.13
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.0.0: Fri Sep 15 14:42:57 PDT 2023; root:xnu-10002.1.13~1/RELEASE_ARM64_T8112
Disk Space Avail:   337.74 GB / 494.38 GB (68.3%)
Train Data Rows:    2044
Train Data Columns: 28
Label Column: salary
Preprocessing data ...
/Users/main/anaconda3/envs/autogluon/lib/

[1000]	valid_set's l1: 3.93782e+06


	-3379640.389	 = Validation score   (-mean_absolute_error)
	14.44s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ... Training model for up to 784.02s of the 1184.19s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
/Users/main/anaconda3/envs/autogluon/lib/python3.10/site-packages/autogluon/common/utils/pandas_utils.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2336.5714285714284' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  memory_usage[column] = (
/Users/main/anaconda3/envs/autogluon/lib/python3.10/site-packages/autogluon/common/utils/pandas_utils.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2336.5714285714284' has dtype incompatible with int64, please explicitly cast to a compatible dtype fir

[1000]	valid_set's l1: 4.06192e+06
[2000]	valid_set's l1: 4.06175e+06
[3000]	valid_set's l1: 4.06175e+06
[4000]	valid_set's l1: 4.06175e+06
[5000]	valid_set's l1: 4.06175e+06
[6000]	valid_set's l1: 4.06175e+06
[7000]	valid_set's l1: 4.06175e+06
[8000]	valid_set's l1: 4.06175e+06
[9000]	valid_set's l1: 4.06175e+06
[10000]	valid_set's l1: 4.06175e+06


	-3409392.6713	 = Validation score   (-mean_absolute_error)
	137.94s	 = Training   runtime
	0.23s	 = Validation runtime
Repeating k-fold bagging: 2/20
Fitting model: LightGBMXT_BAG_L1 ... Training model for up to 571.18s of the 971.35s of remaining time.
	Fitting 8 child models (S2F1 - S2F8) | Fitting with SequentialLocalFoldFittingStrategy
/Users/main/anaconda3/envs/autogluon/lib/python3.10/site-packages/autogluon/common/utils/pandas_utils.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2336.5714285714284' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  memory_usage[column] = (
/Users/main/anaconda3/envs/autogluon/lib/python3.10/site-packages/autogluon/common/utils/pandas_utils.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2336.5714285714284' has dtype incompatible with int64, please explici

...

/Users/main/anaconda3/envs/autogluon/lib/python3.10/site-packages/autogluon/common/utils/pandas_utils.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2336.5714285714284' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  memory_usage[column] = (
/Users/main/anaconda3/envs/autogluon/lib/python3.10/site-packages/autogluon/common/utils/pandas_utils.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2337.5714285714284' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  memory_usage[column] = (
/Users/main/anaconda3/envs/autogluon/lib/python3.10/site-packages/autogluon/common/utils/pandas_utils.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2337.5714285714284' has dtype incompatible with int

[1000]	valid_set's l1: 3.63787e+06


/Users/main/anaconda3/envs/autogluon/lib/python3.10/site-packages/autogluon/common/utils/pandas_utils.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2336.5714285714284' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  memory_usage[column] = (
/Users/main/anaconda3/envs/autogluon/lib/python3.10/site-packages/autogluon/common/utils/pandas_utils.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2336.5714285714284' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  memory_usage[column] = (
/Users/main/anaconda3/envs/autogluon/lib/python3.10/site-packages/autogluon/common/utils/pandas_utils.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2337.5714285714284' has dtype incompatible with int

[1000]	valid_set's l1: 3.00307e+06


/Users/main/anaconda3/envs/autogluon/lib/python3.10/site-packages/autogluon/common/utils/pandas_utils.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2336.5714285714284' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  memory_usage[column] = (
/Users/main/anaconda3/envs/autogluon/lib/python3.10/site-packages/autogluon/common/utils/pandas_utils.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2336.5714285714284' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  memory_usage[column] = (
/Users/main/anaconda3/envs/autogluon/lib/python3.10/site-packages/autogluon/common/utils/pandas_utils.py:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2336.5714285714284' has dtype incompatible with int

In [13]:
predictor = TabularPredictor.load(save_path)# no need just incase path missing
# Test trained model on val/test data set
val_pred = predictor.predict(val_data_noLabel)
perf = predictor.evaluate_predictions(y_true=val_data['salary'], y_pred=val_pred, auxiliary_metrics=True)

/Users/main/anaconda3/envs/autogluon/lib/python3.10/site-packages/autogluon/features/generators/fillna.py:58: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  X.fillna(self._fillna_feature_map, inplace=True, downcast=False)
Evaluation: mean_absolute_error on test data: -3351819.9755381607
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "mean_absolute_error": -3351819.9755381607,
    "root_mean_squared_error": -5365244.765349902,
    "mean_squared_error": -28785851392114.527,
    "r2": 0.7555810710431778,
    "pearsonr": 0.8709215156942515,
    "median_absolute_error": -1753217.0
}


Predicted salary is: 
 694     7.115176e+06
2364    2.647658e+06
885     2.017741e+06
972     7.582161e+05
2155    1.590490e+07
            ...     
2256    6.270718e+06
35      3.605286e+07
1321    5.840409e+05
85      1.037438e+07
1086    1.567242e+07
Name: salary, Length: 511, dtype: float32
val data is 
 694      4633612.0
2364     2416393.0
885      1609941.0
972       109443.0
2155    17271916.0
           ...    
2256     5678291.0
35      32600060.0
1321     2099339.0
85      18154000.0
1086    13700348.0
Name: salary, Length: 511, dtype: float64


In [ ]:
#1st run  we used medium quality model for fast testing. Both rmsq and absolute mean error 
#2nd run, good quality, autostack,num_bag_folds=3 "mean_absolute_error": -3630347,"root_mean_squared_error": -5654671,"mean_squared_error": -31975306135268,"r2": 0.68088,"median_absolute_error": -2755374
#3rd run, feed all 5 years of data to model,      "mean_absolute_error": -2845665,"root_mean_squared_error": -4370963,"mean_squared_error": -19105321713508,"r2": 0.81100,"median_absolute_error": -1652856
#4th run, without name                            "mean_absolute_error": -3351819,root_mean_squared_error": -5365244,"mean_squared_error": -28785851392114, "r2": 0.75558,"median_absolute_error": -1753217

In [24]:
df_submission = pd.DataFrame(columns =["original","prediction"])
df_submission["original"] = val_data['salary']
df_submission["prediction"] = val_pred
print(df_submission)
df_submission.to_csv("original_prediction")

        original    prediction
694    4633612.0  7.115176e+06
2364   2416393.0  2.647658e+06
885    1609941.0  2.017741e+06
972     109443.0  7.582161e+05
2155  17271916.0  1.590490e+07
...          ...           ...
2256   5678291.0  6.270718e+06
35    32600060.0  3.605286e+07
1321   2099339.0  5.840409e+05
85    18154000.0  1.037438e+07
1086  13700348.0  1.567242e+07

[511 rows x 2 columns]


In [14]:
predictor.feature_importance(val_data)

Computing feature importance via permutation shuffling for 28 features using 511 rows with 5 shuffle sets...
/Users/main/anaconda3/envs/autogluon/lib/python3.10/site-packages/autogluon/features/generators/fillna.py:58: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  X.fillna(self._fillna_feature_map, inplace=True, downcast=False)
	558.8s	= Expected runtime (111.76s per shuffle set)
/Users/main/anaconda3/envs/autogluon/lib/python3.10/site-packages/autogluon/features/generators/fillna.py:58: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  X.fillna(self._fillna_feature_map, inplace=True, downcast=False)
/Users/m

,importance,stddev,p_value,n,p99_high,p99_low
Age,1.964843e+06,125353.716976,0.000002,5,2.222948e+06,1.706737e+06
FP,9.286945e+05,70752.181571,0.000004,5,1.074374e+06,7.830148e+05
Min,2.789837e+05,30866.735688,0.000018,5,3.425388e+05,2.154287e+05
TOV,1.663891e+05,35567.971336,0.000236,5,2.396240e+05,9.315413e+04
PTS,1.610344e+05,20488.997823,0.000031,5,2.032215e+05,1.188473e+05
FGA,1.508286e+05,39641.995204,0.000523,5,2.324520e+05,6.920520e+04
FTA,9.047676e+04,18769.487597,0.000210,5,1.291234e+05,5.183012e+04
DREB,7.650400e+04,18100.114709,0.000349,5,1.137724e+05,3.923561e+04
3PA,6.843442e+04,23218.991873,0.001373,5,1.162426e+05,2.062619e+04
FTM,6.802183e+04,23474.286367,0.001462,5,1.163557e+05,1.968795e+04


In [15]:
results = predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                     model     score_val  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0      WeightedEnsemble_L3 -3.116913e+06       1.707507  1013.717373                0.000206           0.113802            3       True         22
1      WeightedEnsemble_L2 -3.142718e+06       0.237016   167.488648                0.000192           0.130868            2       True         12
2    NeuralNetTorch_BAG_L2 -3.164924e+06       1.299138   730.777529                0.186200         120.595402            2       True         20
3          LightGBM_BAG_L1 -3.199807e+06       0.043903    28.306551                0.043903          28.306551            1       True          4
4          CatBoost_BAG_L2 -3.213444e+06       1.154133   633.404181                0.041195          23.222053            2       True         16
5        LightGBMXT_BAG_L2 -3.222414e+06       1.260120 

/Users/main/anaconda3/envs/autogluon/lib/python3.10/site-packages/autogluon/core/utils/plots.py:169: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')


In [16]:
predictor.leaderboard(val_data)

/Users/main/anaconda3/envs/autogluon/lib/python3.10/site-packages/autogluon/features/generators/fillna.py:58: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  X.fillna(self._fillna_feature_map, inplace=True, downcast=False)


                     model    score_test     score_val  pred_time_test  pred_time_val     fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0           XGBoost_BAG_L1 -3.345122e+06 -3.239664e+06        0.153760       0.051922    14.195032                 0.153760                0.051922          14.195032            1       True          9
1      WeightedEnsemble_L3 -3.351820e+06 -3.116913e+06        3.994934       1.707507  1013.717373                 0.001056                0.000206           0.113802            3       True         22
2      WeightedEnsemble_L2 -3.365696e+06 -3.142718e+06        0.456076       0.237016   167.488648                 0.001022                0.000192           0.130868            2       True         12
3          CatBoost_BAG_L1 -3.374262e+06 -3.335811e+06        0.035932       0.038115    25.361893                 0.035932                0.038115          25.361893            1       True  

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,XGBoost_BAG_L1,-3.345122e+06,-3.239664e+06,0.153760,0.051922,14.195032,0.153760,0.051922,14.195032,1,True,9
1,WeightedEnsemble_L3,-3.351820e+06,-3.116913e+06,3.994934,1.707507,1013.717373,0.001056,0.000206,0.113802,3,True,22
2,WeightedEnsemble_L2,-3.365696e+06,-3.142718e+06,0.456076,0.237016,167.488648,0.001022,0.000192,0.130868,2,True,12
3,CatBoost_BAG_L1,-3.374262e+06,-3.335811e+06,0.035932,0.038115,25.361893,0.035932,0.038115,25.361893,1,True,6
4,XGBoost_BAG_L2,-3.376107e+06,-3.239189e+06,2.955240,1.166379,626.087746,0.107278,0.053441,15.905619,2,True,19
5,LightGBMLarge_BAG_L1,-3.384930e+06,-3.339921e+06,1.627304,0.473607,347.283265,1.627304,0.473607,347.283265,1,True,11
6,CatBoost_BAG_L2,-3.403956e+06,-3.213444e+06,2.882417,1.154133,633.404181,0.034455,0.041195,23.222053,2,True,16
7,NeuralNetTorch_BAG_L2,-3.404906e+06,-3.164924e+06,3.090668,1.299138,730.777529,0.242705,0.186200,120.595402,2,True,20
8,RandomForestMSE_BAG_L1,-3.414620e+06,-3.395405e+06,0.066384,0.066767,1.192908,0.066384,0.066767,1.192908,1,True,5
9,NeuralNetFastAI_BAG_L2,-3.430333e+06,-3.293018e+06,3.053564,1.213325,638.972101,0.205601,0.100386,28.789973,2,True,18


In [17]:
predictor.problem_type

'regression'

In [ ]:
matplotlib.pyplot.boxplot()